## Importando as bibliotecas

In [2]:
# pip install basedosdados==2.0.0b16 de acordo com 
import basedosdados as bd

Tive problemas para instalar somente utilizando `pip install basedosdados`, verifiquei na issue ([visualizar issue](https://github.com/basedosdados/mais/issues/1684#issuecomment-1781128338)) que falava sobre uma correção nas dependências no momento da instalação, foi recomendado utilizar **`pip install basedosdados==2.0.0b16`** conforme o **pull** que foi mencionado na issue anterior. ([visualizar pull](https://github.com/basedosdados/mais/pull/1678))

#### 1. Quantos chamados foram abertos no dia 01/04/2023?

In [3]:
# Definimos a query
query1 = """
SELECT 
    COUNT(*) AS chamados
FROM 
    `datario.adm_central_atendimento_1746.chamado`
WHERE 
    DATE(data_inicio) = '2023-04-01'
"""

In [15]:
# o ID do projeto a ser 'cobrado', de acordo com recomendação do github da lib
bd.config.billing_project_id = '<dados-rio-gbs>'

In [4]:
# criando o Pandas DataFrame com a query
df_query1 = bd.read_sql(query1)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=262006177488-3425ks60hkk80fssi9vpohv88g6q1iqd.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=lJI0mhHKqlRTuMF0jw8lt2hSNKmlsY&prompt=consent&access_type=offline
Downloading: 100%|██████████|


In [5]:
# Visualizando a tabela
df_query1.head()

,chamados
0,1756


R: 1756 chamados foram abertoes neste dia.

A resposta saiu conforme esperado nesta tentativa de uso da biblioteca, iremos continuar utilizando este formato.

#### 2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [7]:
# Definimos a query
query2 = """
SELECT 
    tipo, COUNT(*) AS chamadas_por_tipo 
FROM 
    `datario.adm_central_atendimento_1746.chamado`
WHERE 
    DATE(data_inicio) = '2023-04-01'
GROUP BY 
    tipo
ORDER BY 
    chamadas_por_tipo DESC
"""

# criando o Pandas DataFrame
df_query2 = bd.read_sql(query2, billing_project_id="dados-rio-gbs")
df_query2.head()

Downloading: 100%|██████████|


,tipo,chamadas_por_tipo
0,Estacionamento irregular,366
1,Remoção Gratuita,196
2,Manutenção de iluminação pública,122
3,Limpeza de logradouros,116
4,Perturbação do sossego,111


R: Estacionamento irregular com 366 chamados abertos.

#### 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [8]:
query3 = """
SELECT 
  `datario.adm_central_atendimento_1746.chamado`.id_bairro,
  `datario.dados_mestres.bairro`.nome as nome_bairro,
  COUNT(*) AS qtd_chamado 
FROM 
  `datario.adm_central_atendimento_1746.chamado`
LEFT JOIN 
  `datario.dados_mestres.bairro`
  ON `datario.adm_central_atendimento_1746.chamado`.id_bairro = `datario.dados_mestres.bairro`.id_bairro
WHERE
  DATE(data_inicio) = '2023-04-01'
GROUP BY
  id_bairro, nome_bairro
ORDER BY
  qtd_chamado DESC
"""

# criando o Pandas DataFrame
df_query3 = bd.read_sql(query3, billing_project_id="dados-rio-gbs")
df_query3.head()

Downloading: 100%|██████████|


,id_bairro,nome_bairro,qtd_chamado
0,144,Campo Grande,113
1,33,Tijuca,89
2,None,None,73
3,128,Barra da Tijuca,59
4,141,Bangu,53


R: Campo Grande, Tijuca e Barra da Tijuca. Importante salientar que há 73 chamados não associados a um bairro, por isso o bairro 128 foi incluído no top 3.

#### 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [9]:
query4 = """
SELECT 
  `datario.dados_mestres.bairro`.subprefeitura,
  COUNT(*) AS qtd_chamado 
FROM 
  `datario.adm_central_atendimento_1746.chamado`
LEFT JOIN 
  `datario.dados_mestres.bairro`
  ON `datario.adm_central_atendimento_1746.chamado`.id_bairro = `datario.dados_mestres.bairro`.id_bairro
WHERE
  DATE(data_inicio) = '2023-04-01'
GROUP BY
  subprefeitura
ORDER BY
  qtd_chamado DESC
"""

# criando o Pandas DataFrame
df_query4 = bd.read_sql(query4, billing_project_id="dados-rio-gbs")
df_query4.head()

Downloading: 100%|██████████|


,subprefeitura,qtd_chamado
0,Zona Norte,510
1,Zona Oeste,238
2,Zona Sul,233
3,Grande Tijuca,159
4,Jacarepaguá,145


R: subprefeitura 'Zona Norte', com 510 chamados abertos.

#### 5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [10]:
query5 = """
SELECT 
  `datario.dados_mestres.bairro`.nome as nome_bairro,
  `datario.dados_mestres.bairro`.subprefeitura,
  COUNT(*) AS qtd_chamado 
FROM 
  `datario.adm_central_atendimento_1746.chamado`
LEFT JOIN 
  `datario.dados_mestres.bairro`
  ON `datario.adm_central_atendimento_1746.chamado`.id_bairro = `datario.dados_mestres.bairro`.id_bairro
WHERE
  DATE(data_inicio) = '2023-04-01'
GROUP BY
  nome_bairro, subprefeitura
ORDER BY
  qtd_chamado DESC
"""

# criando o Pandas DataFrame
df_query5 = bd.read_sql(query5, billing_project_id="dados-rio-gbs")
df_query5.head()

Downloading: 100%|██████████|


,nome_bairro,subprefeitura,qtd_chamado
0,Campo Grande,Zona Oeste,113
1,Tijuca,Grande Tijuca,89
2,None,None,73
3,Barra da Tijuca,Barra da Tijuca,59
4,Bangu,Grande Bangu,53


R: Sim, existem 73 chamados abertos que não foram associados a um bairro ou subprefeitura.

Pode ocorrer por dois motivos: 

1. Erro de digitação/inserção; ou
2. A informação de bairro/subprefeitura foi corrompida pelo próprio código do sistema que o insere.

#### 6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [12]:
query6 = """
SELECT 
  subtipo, COUNT(*) as chamados_por_subtipo
FROM 
  `datario.adm_central_atendimento_1746.chamado`
WHERE 
  DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31' AND subtipo = 'Perturbação do sossego'
GROUP BY 
  subtipo
"""

# criando o Pandas DataFrame
df_query6 = bd.read_sql(query6, billing_project_id="dados-rio-gbs")
df_query6.head()

Downloading: 100%|██████████|


,subtipo,chamados_por_subtipo
0,Perturbação do sossego,42830


R: 42.830 chamados abertos no período acima mencionado, incluindo extremidades.

#### 7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

R: Query.

In [16]:
query7 = """
SELECT 
  *
FROM 
  `datario.adm_central_atendimento_1746.chamado`
RIGHT JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
  ON DATE(`datario.adm_central_atendimento_1746.chamado`.data_inicio) 
    BETWEEN 
      DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_inicial) AND DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_final)
WHERE 
  subtipo = 'Perturbação do sossego'
"""
  
# criando o Pandas DataFrame
df_query7 = bd.read_sql(query7, billing_project_id="dados-rio-gbs")
df_query7.head()

Downloading: 100%|██████████|


,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,ano,data_inicial,data_final,evento,taxa_ocupacao
0,18330689,11,2023-02-19 16:02:04,2023-02-19 16:13:01,90,3,10363,94,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Atendido,None,1,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
1,18329490,11,2023-02-18 21:10:10,2023-02-24 10:52:33,128,4,198499,<NA>,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
2,18330160,11,2023-02-19 10:28:51,2023-02-27 08:56:48,5,1,151415,84,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
3,18331102,11,2023-02-19 20:55:17,2023-02-28 10:04:05,90,3,34868,243,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
4,18329699,1,2023-02-19 00:08:53,2023-02-27 08:34:28,147,5,298851,0,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,Encerrado,Não atendido,None,0,2023-02-01,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554


In [ ]:
df_query7.shape

#### 8. Quantos chamados desse subtipo foram abertos em cada evento?

In [17]:
query8 = """
SELECT 
  evento,
  COUNT(*) as chamados_por_evento
FROM 
  `datario.adm_central_atendimento_1746.chamado`
RIGHT JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
  ON DATE(`datario.adm_central_atendimento_1746.chamado`.data_inicio) 
    BETWEEN 
      DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_inicial) AND DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_final)
WHERE 
  subtipo = 'Perturbação do sossego'
GROUP BY 
  evento
"""

# criando o Pandas DataFrame
df_query8 = bd.read_sql(query8, billing_project_id="dados-rio-gbs")
df_query8.head()

Downloading: 100%|██████████|


,evento,chamados_por_evento
0,Rock in Rio,834
1,Carnaval,241
2,Reveillon,139


R: 834 chamados durante os dois períodos do Rock in Rio, 241 chamados durante o Carnaval e 139 chamados durante o Reveillon.

#### 9. Qual evento teve a maior média diária de chamados abertos desse subtipo?

Para isso, criarei uma CTE e utilizarei a média da contagem por dia de evento.

In [21]:

query9 = """
WITH diarias AS (
  SELECT
    evento,
    DATE(data_inicio) AS data_chamado,
    COUNT(*) AS chamados_diarios
  FROM
    `datario.adm_central_atendimento_1746.chamado`
  RIGHT JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
    ON DATE(`datario.adm_central_atendimento_1746.chamado`.data_inicio) BETWEEN DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_inicial) AND DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_final)
  WHERE
    subtipo = 'Perturbação do sossego'
  GROUP BY
    evento,
    DATE(data_inicio)
)
SELECT
  evento,
  AVG(chamados_diarios) AS media_chamados_por_evento
FROM
  diarias
GROUP BY
  evento"""
  
# criando o Pandas DataFrame
df_query9 = bd.read_sql(query9, billing_project_id="dados-rio-gbs")
df_query9.head()

Downloading: 100%|██████████|


,evento,media_chamados_por_evento
0,Rock in Rio,119.142857
1,Carnaval,60.250000
2,Reveillon,46.333333


R: Rock in Rio juntando os dois períodos com 119 chamados abertos por dia de evento.

#### 10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [22]:
query10 = """
WITH diarias AS (
  SELECT
    evento,
    DATE(data_inicio) AS data_chamado,
    COUNT(*) AS chamados_diarios
  FROM
    `datario.adm_central_atendimento_1746.chamado`
  RIGHT JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
    ON DATE(`datario.adm_central_atendimento_1746.chamado`.data_inicio) BETWEEN DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_inicial) AND DATE(`datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`.data_final)
  WHERE
    subtipo = 'Perturbação do sossego'
  GROUP BY
    evento,
    DATE(data_inicio)
),
media_chamados_por_evento AS (
SELECT
  evento,
  AVG(chamados_diarios) AS media_chamados_por_evento
FROM
  diarias
GROUP BY
  evento
),
chamados_diarios_ano AS (
  SELECT
    DATE(data_inicio) AS data_chamada,
    COUNT(*) AS chamados_diarios
  FROM
    `datario.adm_central_atendimento_1746.chamado`
  WHERE
    DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
    AND subtipo = 'Perturbação do sossego'
  GROUP BY
    DATE(data_inicio)
),
media_chamados_ano AS (
SELECT
  AVG(chamados_diarios) AS media_diaria_chamados
FROM
  chamados_diarios_ano
)
SELECT
  m.evento,
  m.media_chamados_por_evento,
  d.media_diaria_chamados
FROM
  media_chamados_por_evento AS m
CROSS JOIN
  media_chamados_ano AS d
  """
  
# criando o Pandas DataFrame
df_query10 = bd.read_sql(query10, billing_project_id="dados-rio-gbs")
df_query10.head()

Downloading: 100%|██████████|


,evento,media_chamados_por_evento,media_diaria_chamados
0,Reveillon,46.333333,61.982634
1,Rock in Rio,119.142857,61.982634
2,Carnaval,60.250000,61.982634


R: Fiz CTE's com as queries das questões anteriores e utilize um CROSS JOIN para comparar os períodos, como consta acima.

A proporção de chamados abertos do período mencionado em relação ao período dos eventos é parecida exceto pelo evento Rock in Rio que possui quase o dobro de chamados.